# UMAP Implementation on Burst features and windows

In [1]:
from ipyfilechooser import FileChooser
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import StandardScaler
import umap

In [ ]:
fc = FileChooser('./')
fc.title = "<b>Select preprocessing parameter file</b>"
fc.filter_pattern = ['*.npz']
display(fc)

FileChooser(path='C:\Users\omgui\Desktop\BASUS', filename='', title='<b>Select preprocessing parameter file</b…

In [15]:
path = r'C:\Users\omgui\Desktop\BASUS\sim_data\TentSim_v1p9p3_23Jun25-114549\Pre_v1p1_23Jun25-121748\Pre_v1p123Jun25-114549-preparams.par23Jun25-121748.npz'

In [16]:
data = np.load(path, allow_pickle=True)
windows = np.array(data['handlesA'].item()['burst_windows'])
labels = data['handlesA'].item()['species']
assert windows.shape[0] == labels.shape[0]
windows.shape

(796, 13)

In [17]:
amplitudes = data['handlesA'].item()['amplitude']
widths = data['handlesA'].item()['width']
rises = data['handlesA'].item()['rise']
falls = data['handlesA'].item()['fall']
skews = data['handlesA'].item()['skew']
kurts = data['handlesA'].item()['kurt']
ibis = data['handlesA'].item()['ibis']
assert amplitudes.shape == widths.shape == rises.shape == falls.shape == skews.shape == kurts.shape == labels.shape
assert (labels == -1).sum() == 0
df = pd.DataFrame({
    'Amplitude':amplitudes,
    'Width':widths,
    'IBIs':ibis,
    'Labels':labels,
    #'Area':areas,
    'RiseTime':rises,
    'FallTime':falls,
    'Skewness':skews,
    'Kurtosis':kurts,
})

In [18]:
windows = data['handlesA'].item()['burst_windows']
windows

array([[ 3.14202434,  0.        ,  1.14202485, ...,  0.14202716,
         0.        ,  2.14202779],
       [ 2.14219996,  0.        ,  9.14220289, ...,  5.14221528,
         3.14221686,  0.        ],
       [ 0.        ,  0.        ,  2.14252383, ...,  0.        ,
         0.14254502,  5.1425474 ],
       ...,
       [ 2.07750513,  6.07751455,  4.07752395, ..., 12.0775988 ,
         2.07760811,  0.        ],
       [ 2.07833467,  1.07834303,  1.07835139, ...,  4.07841792,
         0.        ,  0.0784344 ],
       [ 1.08100592,  2.08100626,  0.08100665, ...,  0.08100875,
        12.08100894, 18.08100911]], shape=(796, 13))

In [19]:
features = df.drop(columns=['Labels'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [36]:
reducer = umap.UMAP(n_components=3, random_state=42, n_neighbors=100, min_dist=0.0001)
#embedding = reducer.fit_transform(X_scaled)
embedding = reducer.fit_transform(windows)

c:\Users\omgui\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\omgui\AppData\Local\Programs\Python\Python313\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [37]:
df_umap = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2', 'UMAP3'])
df_umap['Species'] = df['Labels'].values

fig = px.scatter_3d(
    df_umap,
    x='UMAP1',
    y='UMAP2',
    z='UMAP3',
    color='Species',
    title='UMAP for burst windows',
    labels={'Species': 'Burst Type'},
    width=800,
    height=700
)

fig.update_traces(marker=dict(size=5))
fig.show()

embedding = reducer.fit_transform(X_scaled)
df_umap = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2', 'UMAP3'])
df_umap['Species'] = df['Labels'].values

fig = px.scatter_3d(
    df_umap,
    x='UMAP1',
    y='UMAP2',
    z='UMAP3',
    color='Species',
    title='UMAP for burst features',
    labels={'Species': 'Burst Type'},
    width=800,
    height=700
)

fig.update_traces(marker=dict(size=5))
fig.show()

c:\Users\omgui\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [38]:
# 1. Select the 7 numerical features (exclude 'Labels')
feature_cols = ['Amplitude', 'Width', 'RiseTime', 'FallTime', 'Skewness', 'Kurtosis', 'IBIs']
features = df[feature_cols].values  # shape (N, 7)

# 2. Combine windows and features
combined_data = np.concatenate([windows, features], axis=1)  # shape (N, 20)

# 3. Run UMAP on the combined data
import umap
reducer = umap.UMAP(n_components=3, random_state=42)
embedding = reducer.fit_transform(combined_data)

# 4. Plot
import plotly.express as px
df_umap = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2', 'UMAP3'])
df_umap['Species'] = df['Labels'].values

fig = px.scatter_3d(
    df_umap,
    x='UMAP1',
    y='UMAP2',
    z='UMAP3',
    color='Species',
    title='UMAP: Burst Windows + Features',
    labels={'Species': 'Burst Type'},
    width=800,
    height=700
)
fig.update_traces(marker=dict(size=5))
fig.show()

c:\Users\omgui\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\omgui\AppData\Local\Programs\Python\Python313\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

